In [1]:
# import os, sys
# from google.colab import drive
# drive.mount('/content/mnt', force_remount=True)
# nb_path = '/content/notebooks'
# os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
# sys.path.insert(0, nb_path)  # or append(nb_path)

In [2]:
import rom_operator_inference as opinf

import pickle
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error as mse
from torch.utils.data import DataLoader
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np


PREFIX = "."

In [3]:
# !pip install git+https://github.com/Willcox-Research-Group/rom-operator-inference-Python3.git

according to the https://github.com/Willcox-Research-Group/rom-operator-inference-Python3/blob/main/docs/content/tutorials/basics.ipynb, the matrix must be $space \times time$

In [4]:
with open(f'{PREFIX}/train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

with open(f'{PREFIX}/val_data.pkl', 'rb') as f:
    val_data = pickle.load(f)

with open(f'{PREFIX}/test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

In [5]:
# pure copy
N_COMPONENTS = 12
INPUT_STEP = 10
OUTPUT_STEP = 1
BATCH_SIZE = 8
DELTA_T = 1e-4 # step of a time

In [6]:
train_data = train_data.T
val_data = val_data.T
test_data = test_data.T

In [7]:
assert train_data.shape[0] > train_data.shape[1]   # space is larger than time

In [154]:
def prepare_data_and_train(data_to_train, r):
    train_data_derivative = opinf.pre.ddt(data_to_train, DELTA_T, order=6)        # Calculate the time derivative matrix.
    Vr, _ = opinf.pre.pod_basis(data_to_train, r=r)    
    rom = opinf.ContinuousOpInfROM(modelform="cA")
    rom.fit(Vr, data_to_train, train_data_derivative)
    return rom

In [155]:
from tqdm.notebook import tqdm

best_basis, best_basis_val = 0, float('inf')
for r in tqdm(range(1,32)):
    rom = prepare_data_and_train(train_data, r)
    prediction = rom.predict(train_data[:,-1], np.arange(0.0499, 0.0597, 1e-4))   # predict from the last column of train data
    tmp = mse(prediction, val_data, squared=False)   
    if tmp < best_basis_val:
        best_basis_val = tmp
        best_basis = r

  0%|          | 0/31 [00:00<?, ?it/s]

In [156]:
print(f"With r={best_basis}, we have best result, which is {best_basis_val}")

With r=12, we have best result, which is 16559.189606957672


## Train on train, val, and predict test

In [159]:
to_train = np.concatenate((train_data, val_data), axis=1)
rom = prepare_data_and_train(to_train, best_basis)

In [166]:
prediction = rom.predict(val_data[:,-1], np.arange(0.0598, 0.0999, 1e-4))   # predict from the last column of val data
tmp = mse(prediction, test_data, squared=False)

In [168]:
tmp

530.5494631954839